# Script APA para la detección de vegetación acuática y algas con Sentinel-2

[Script APA de detección de vegetación acuática y algas con Sentinel-2](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/apa_script/)

In [1]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt

from rasterio.warp import reproject, Resampling
from pathlib import Path

In [2]:
# Definición de rutas de archivo
base_path = "sentinel2_b02.tif"  # banda de referencia (10 m)
align_paths = [
    "sentinel2_b03.tif",  # 10 m
    "sentinel2_b04.tif",  # 10 m
    "sentinel2_b05.tif",  # 20 m
    "sentinel2_b08.tif",  # 10 m
    "sentinel2_b11.tif",  # 20 m
]

In [3]:
def load_and_align_bands(base_path, align_paths, scale=0.0001,
                         resampling=Resampling.bilinear):

    # Banda de referencia
    with rasterio.open(base_path) as src:
        base = src.read(1).astype(np.float32)
        nodata = src.nodata
        if nodata is not None:
            base = np.where(base == nodata, np.nan, base)
        base *= scale  # aplicar factor de escala
        base_profile = src.profile

    aligned = {}

    for p in align_paths:
        stem = Path(p).stem

        with rasterio.open(p) as src:
            arr = src.read(1).astype(np.float32)
            prof = src.profile
            nodata = src.nodata

        # Manejo de valores nodata
        if nodata is not None:
            arr = np.where(arr == nodata, np.nan, arr)

        # Aplicar factor de escala
        arr *= scale

        # Remuestreo si es necesario
        if (prof["transform"] != base_profile["transform"] or
            prof["crs"] != base_profile["crs"] or
            prof["height"] != base_profile["height"] or
            prof["width"] != base_profile["width"]):

            dst = np.empty((base_profile["height"], base_profile["width"]),
                           dtype=np.float32)

            reproject(
                source=arr,
                destination=dst,
                src_transform=prof["transform"],
                src_crs=prof["crs"],
                dst_transform=base_profile["transform"],
                dst_crs=base_profile["crs"],
                resampling=resampling
            )

            arr = dst

        aligned[stem] = arr

    return base, aligned, base_profile

In [4]:
B02, bands, base_profile = load_and_align_bands(base_path, align_paths)
B03 = bands["sentinel2_b03"]
B04 = bands["sentinel2_b04"]
B05 = bands["sentinel2_b05"]
B08 = bands["sentinel2_b08"]
B11 = bands["sentinel2_b11"]

In [5]:
def compute_aquatic_indices_rgb(B02, B03, B04, B05, B08, B11):
    """
    Calcula los índices espectrales para la detección de vegetación acuática y algas
    (NDWI, water_plants, FAI) y genera un RGB acuático.
    """

    # División segura
    def safe_index(num, den):
        num = num.astype(np.float32)
        den = den.astype(np.float32)
        with np.errstate(divide="ignore", invalid="ignore"):
            out = num / den
        mask = (den == 0) | np.isnan(num) | np.isnan(den)
        out[mask] = np.nan
        return out.astype(np.float32)

    # Convertir todas las bandas a float32
    B02 = B02.astype(np.float32)
    B03 = B03.astype(np.float32)
    B04 = B04.astype(np.float32)
    B05 = B05.astype(np.float32)
    B08 = B08.astype(np.float32)
    B11 = B11.astype(np.float32)

    # Cálculo de índices espectrales
    NDWI = safe_index(B03 - B08, B03 + B08)          # Índice de detección de agua
    water_plants = safe_index(B05 - B04, B05 + B04)  # Índice de vegetación acuática

    # Floating Algae Index (FAI)
    NIR2 = B04 + (B11 - B04) * ((832.8 - 664.6) / (1613.7 - 664.6))
    FAI = B08 - NIR2                                 # Índice de algas flotantes

    # Composición RGB acuática
    rgb = np.stack([
        8.5 * FAI,
        5.5 * water_plants,
        1.0 * NDWI,
    ], axis=0).astype(np.float32)

    return rgb, NDWI, FAI, water_plants

In [6]:
rgb, NDWI, FAI, water_plants = compute_aquatic_indices_rgb(
    B02, B03, B04, B05, B08, B11
)

In [7]:
def save_single_band(arr, profile, path, nodata_value=-9999.0):
    """
    Guarda una sola banda como GeoTIFF.
    Los valores NaN se registran como nodata_value.
    """
    arr_to_save = np.where(np.isnan(arr), nodata_value, arr).astype(np.float32)

    prof = profile.copy()
    prof.update(
        dtype=rasterio.float32,
        count=1,
        compress="lzw",
        nodata=nodata_value,
    )

    with rasterio.open(path, "w", **prof) as dst:
        dst.write(arr_to_save, 1)

    print("Guardado:", path)

In [8]:
# Guardar cada índice individual como GeoTIFF
save_single_band(NDWI, base_profile, "ndwi.tif")
save_single_band(FAI, base_profile, "fai.tif")
save_single_band(water_plants, base_profile, "water_plants.tif")

Guardado: ndwi.tif
Guardado: fai.tif
Guardado: water_plants.tif


In [9]:
def save_rgb(
    rgb,
    profile,
    path="aquatic_rgb.tif",
    nodata_value=-9999.0,
    band_names=["FAI", "WATER_PLANTS", "NDWI"]
):
    """
    Guarda un arreglo RGB (3, H, W) como un GeoTIFF de 3 bandas.
    Los valores NaN se reemplazan por nodata_value y también se guardan
    los nombres de las bandas.
    """
    rgb_to_save = np.where(np.isnan(rgb), nodata_value, rgb).astype(np.float32)

    prof = profile.copy()
    prof.update(
        dtype=rasterio.float32,
        count=3,
        compress="lzw",
        nodata=nodata_value,
    )

    with rasterio.open(path, "w", **prof) as dst:
        # Escribir los datos banda por banda
        for i in range(3):
            dst.write(rgb_to_save[i], i + 1)

        # Agregar descripciones de banda
        for i, name in enumerate(band_names):
            dst.set_band_description(i + 1, name)

    print("Guardado:", path)

In [10]:
# Guardar el RGB acuático como GeoTIFF
save_rgb(rgb, base_profile, "aquatic_rgb.tif")

Guardado: aquatic_rgb.tif


In [11]:
def apply_scl_mask(rgb_path, scl_path, out_path,
                   exclude_classes=[0, 1, 3, 8, 9, 10],
                   nodata_value=-9999):

    # Leer el RGB
    with rasterio.open(rgb_path) as src_rgb:
        rgb = src_rgb.read().astype(np.float32)   # shape: (C, H, W)
        profile = src_rgb.profile
        rgb_transform = src_rgb.transform
        rgb_crs = src_rgb.crs
        height, width = src_rgb.height, src_rgb.width

        # Obtener los nombres (descripciones) de las bandas
        band_descriptions = src_rgb.descriptions

    # Leer el SCL
    with rasterio.open(scl_path) as src_scl:
        scl = src_scl.read(1).astype(np.uint8)
        scl_transform = src_scl.transform
        scl_crs = src_scl.crs
        scl_height, scl_width = src_scl.height, src_scl.width

    # Reproyectar SCL al grid del RGB (nearest, manteniendo valores de clase)
    if (scl_transform != rgb_transform or
        scl_crs != rgb_crs or
        scl_height != height or
        scl_width != width):

        scl_reproj = np.empty((height, width), dtype=np.uint8)

        reproject(
            source=scl,
            destination=scl_reproj,
            src_transform=scl_transform,
            src_crs=scl_crs,
            dst_transform=rgb_transform,
            dst_crs=rgb_crs,
            resampling=Resampling.nearest
        )

        scl = scl_reproj

    # Crear máscara (True = píxel inválido)
    mask = np.isin(scl, exclude_classes)

    # Actualizar perfil con nodata
    profile.update(
        dtype="float32",
        nodata=nodata_value
    )

    # Aplicar la máscara
    rgb[:, mask] = nodata_value

    # Guardar + restaurar nombres de bandas
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(rgb)
        for i, desc in enumerate(band_descriptions, start=1):
            if desc:
                dst.set_band_description(i, desc)

    print("Guardado:", out_path)

In [13]:
# Aplicar la máscara SCL al RGB acuático
apply_scl_mask(
    rgb_path="aquatic_rgb.tif",
    scl_path="sentinel2_scl.tif",
    out_path="aquatic_rgb_masked.tif"
)

Guardado: aquatic_rgb_masked.tif
